In [ ]:
'''
Prerequisites:

Libraries: PyEDFlib, scipy, pandas, numpy
Files: TUH Seizure sub corpus
       TUH_manifest_final.csv with Seizure Onset Zone labels  
       utils_preprocess.py
        
''' 


In [1]:
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import os
import pandas as pd
import csv
import pyedflib
import json
import scipy

from utils_preprocess import *

/home/deeksha/anaconda3/envs/deepai/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup

In [2]:
#Edit: Enter the location of raw data dowloaded from 
#https://isip.piconepress.com/projects/tuh_eeg/html/downloads.shtml


data_fol = '/home/deeksha/TUH/tuh_seizures/'
manifest = read_manifest('TUH_manifest_final.csv', ',')    #edit location if required
print("# Recordings: ", len(manifest))

In [ ]:
#update manifest "loc" with data_fol
updated_mn = manifest.copy()
for i, mnitem in enumerate(manifest):
    oldloc =  mnitem['loc']
    newloc = data_fol + '/' + ('/').join(oldloc.split('/')[5:])
    updated_mn[i]['loc'] = newloc
    
df = pd.DataFrame(data=updated_mn, columns=list(updated_mn[0].keys()))
df.to_csv('data/TUH_manifest_final.csv')   #edit if required

In [6]:
# sanity check for sampling rate and number of seizures per recording
manifest = read_manifest('data/TUH_manifest_final.csv', ',')
for mnitem in manifest:
    loc = mnitem['loc']
    data, fs = readEDF(loc, json.loads(mnitem['nsamples']))
    if fs.min() != fs.max():
            print(mnitem['pt'], fs)
    
    break
    
for mnitem in manifest:
    if mnitem['nsz'] == 0:
        print(mnitem['fn'])

## Preprocess

Resample, filter, crop, and create new manifest file. Save to disk at "newloc"

In [10]:
#Edit: Change the newloc to required final destination
manifest = read_manifest('data/TUH_manifest_final.csv', ',')
newloc = 'data/tuh_preprocessed/'
if not os.path.exists(newloc):
    os.mkdir(newloc)
new_csvdf = manifest.copy()

In [ ]:
fsnew = 200
all_data = []
window_len = 1.
overlap = 0.
for i,mnitem in enumerate(manifest):
    print(i)
    data, fs = readEDF(mnitem['loc'], int(mnitem['nsamples']))
    data_new = []
    for j in range(data.shape[0]):
        data_resampled = resample_eeg(data[j,:], np.float(fs[0]), json.loads(mnitem['nsamples']), fsnew)
        data_preprocessed = preprocess(data_resampled, fsnew)
        data_new.append(data_preprocessed)
    data_new = np.array(data_new)
    
    new_csvdf[i]['loc'] = newloc +mnitem['loc'].split('/')[-1].split('.')[0]+'_win.npy'
    new_csvdf[i]['nsamples'] = data_new.shape[1]
    new_csvdf[i]['fs'] = fsnew
    new_csvdf[i]['duration'] = np.around(data_new.shape[1]/fsnew, 3)
    new_csvdf[i]['nchns'] = data_new.shape[0]
    new_csvdf[i]['fn'] = new_csvdf[i]['fn'].split('.')[0]+'.npy'
    all_window_label = create_label(new_csvdf[i]['duration'],  #max_dur
                                   json.loads(new_csvdf[i]['sz_starts']), 
                                   json.loads(new_csvdf[i]['sz_ends']), 
                                   window_len, overlap)
    windowed_rec = applyWindows(data_new, 
                               fsnew,
                           new_csvdf[i]['duration'], #max_dur
                                window_len, overlap)
    
    np.save(new_csvdf[i]['loc'], windowed_rec)
    np.save(newloc+mnitem['loc'].split('/')[-1].split('.')[0]+'_label.npy', all_window_label)


In [23]:
df = pd.DataFrame(data=new_csvdf, columns=list(new_csvdf[0].keys()))
df.to_csv('data/tuh_windowed_manifest.csv')   #edit if required

200

# Crop and reshape recordings to one seizure per recording

Post the previous preprocessing steps, further crop into single seizure per array

In [79]:
arr_fol =  'data/tuh_preprocessed/'      #edit if required
filenames = os.listdir(arr_fol)
manifest = read_manifest('data/tuh_windowed_manifest.csv', ',')   #edit if required
new_loc = 'data/tuh_windowed/'      #edit if required

In [85]:
maxtot = 600    #ten minutes of recordings
maxpre, maxpost = 300, 300
minpre = 120
minpost=30
newmanifest = []
for i, mnitem in enumerate(manifest):
        print(i)
        xfn = mnitem['loc']
        yfn = xfn.split('.')[0]+'_label.npy'
        y = np.load(yfn)
        fullarr = np.load(xfn)
        #do it for label array also 
        starts = np.ceil(json.loads(mnitem['sz_starts'])+ [fullarr.shape[0]])
        ends = np.ceil(json.loads(mnitem['sz_ends']))
        nsz = json.loads(mnitem['nsz'])
        #    if nsz>0:
        newarr = np.zeros((nsz, maxtot, 19, 200))
        newlabel = np.zeros((nsz, maxtot))
        for i in range(nsz):
            if i==0:
                s = 0
                e = min(int(starts[i+1]-1), int((ends[i] + starts[i+1])/2))
            elif i==nsz-1:
                e = int(starts[i+1]-1)
                s = int((ends[i-1]+starts[i])/2)  
            else:
                s = int((ends[i-1]+starts[i])/2)
                e = min(int(starts[i+1]-1), int((ends[i] + starts[i+1])/2))
                
            totlen = e - s + 1
            szlen = ends[i] - starts[i] + 1
            preseiz = starts[i]-s
            postseiz = e - ends[i]            
                       
            if totlen<=maxtot:
                padlen = int(maxtot-totlen)
                #print('i am so dead', totlen, padlen,s,e, fullarr.shape)
                newarr[i, :,:, :] = np.pad(fullarr[s:e+1, :,:], ((0,padlen),(0,0),(0,0)))
                newlabel[i, :] = np.pad(y[s:e+1], (0,padlen))
              
                #update manifest
                
            elif szlen>=maxtot:
                s = max(int(starts[i]-maxpre), 0)
                newarr[i,:,:,:] = fullarr[s:s+maxtot, :, :]
                newlabel[i,:] = y[s:s+maxtot]
      
                #update mainfest
            else:
                #extra = totlen - maxlen
                preve = ends[i-1] if i>0 else 0
                news = max(preve, starts[i]-minpre)
                newe = min(ends[i]+minpost, starts[i+1]-1)
                newtotlen = newe-news +1
                
                if newtotlen<=maxtot:
                    news = max(preve, int(news-maxtot+newtotlen))
                    newtotlen = newe-news+1
                    if newtotlen<maxtot:
                        newe = min(starts[i+1], int(newe+maxtot-newtotlen))
                        newtotlen = newe-news+1
                    #print(totlen, szlen, s,e,preseiz, postseiz, news, newe )   
                    newarr[i, :,:, :] = fullarr[int(news):int(newe+1), :,:]
                    newlabel[i, :,] = y[int(news):int(newe+1)]
                else:
                    newarr[i, :,:, :] = fullarr[int(news):int(news+maxtot), :,:]
                    newlabel[i, :] = y[int(news):int(news+maxtot)]
                    
                #update manifest what all needs to be changed?
        xfnnew = newloc+xfn.split('/')[-1]
        newmn = mnitem.copy()
        newmn['loc'] = xfnnew
        newmn['duration'] = maxtot
        assert newarr.shape[1] == maxtot == newlabel.shape[1]
        
        np.save(xfnnew, newarr)
        np.save(newloc+yfn.split('/')[-1], newlabel)
        newmanifest.append(newmn)
df = pd.DataFrame(data=newmanifest, columns=list(manifest[0].keys()))
df.to_csv('data/tuh_single_windowed_manifest.csv')
#this final csv is needed in training and validating in the future

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
#